# Tratamiento de datos previo a ML

TFG Predicción de resultados de Fórmula 1

En este notebook se realiza la transformaación de los datos obtenidos de [Ergast API](http://ergast.com/mrd/) con el objetivo de prepararlos para realizar machine learning.

Autor: Manuel Ventura

In [1]:
import numpy as np
import pandas as pd

## Carga de datos

In [2]:
circuits = pd.read_csv('f1db_csv/circuits.csv', na_values=["\\N"])
constructor_results = pd.read_csv('f1db_csv/constructor_results.csv', na_values=["\\N"])
constructor_standings = pd.read_csv('f1db_csv/constructor_standings.csv', na_values=["\\N"])
constructors = pd.read_csv('f1db_csv/constructors.csv', na_values=["\\N"])
driver_standings = pd.read_csv('f1db_csv/driver_standings.csv', na_values=["\\N"])

drivers = pd.read_csv('f1db_csv/drivers.csv', na_values=["\\N"], parse_dates=['dob'])
drivers['number'] = drivers['number'].astype('Int64')

lap_times = pd.read_csv('f1db_csv/lap_times.csv', na_values=["\\N"])
pit_stops = pd.read_csv('f1db_csv/pit_stops.csv', na_values=["\\N"])
qualifying = pd.read_csv('f1db_csv/qualifying.csv', na_values=["\\N"])

date_parse_list = ['date', 'fp1_date', 'fp2_date', 'fp3_date', 'quali_date', 'sprint_date']
races = pd.read_csv('f1db_csv/races.csv', na_values=["\\N"], parse_dates=date_parse_list)

results = pd.read_csv('f1db_csv/results.csv', na_values=["\\N"])
results['number'] = results['number'].astype('Int64')

seasons = pd.read_csv('f1db_csv/seasons.csv', na_values=["\\N"])

sprint_results = pd.read_csv('f1db_csv/sprint_results.csv', na_values=["\\N"])
sprint_results['number'] = sprint_results['number'].astype('Int64')

status = pd.read_csv('f1db_csv/status.csv', na_values=["\\N"])

## Obtener datos de tiempo

In [3]:
import requests
from selenium import webdriver

In [4]:
weather = races.iloc[:,[0]]
weather

,raceId
0,1
1,2
2,3
3,4
4,5
...,...
1097,1116
1098,1117
1099,1118
1100,1119


In [5]:
# Code fom: https://towardsdatascience.com/formula-1-race-predictor-5d4bfae887da

info = []

# read wikipedia tables

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        
                        # find weather in italian with selenium
                        
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima) 
                                
    except:
        info.append('not found')

In [6]:
# append column with weather information to dataframe  

info_ser = pd.Series(info)
weather = weather.merge(info_ser.rename('weather'), left_index=True, right_index=True)

# set up a dictionary to convert weather information into keywords

weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

# map new df according to weather dictionary

weather_df = pd.DataFrame(columns = weather_dict.keys())
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)
   
weather_info = pd.concat([weather, weather_df], axis = 1)
weather_info = weather_info.replace('not found', np.nan)

In [7]:
weather_info.to_csv('f1db_csv/weather.csv', index=False)

In [8]:
weather_info = pd.read_csv('f1db_csv/weather.csv')
weather_info

,raceId,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1,Sunny with temperatures reaching up to 27 °C (...,1,0,0,0,0
1,2,"Dry start, with heavy rain and thunderstorm/mo...",0,0,1,1,0
2,3,Rain,0,0,0,1,0
3,4,Sunny,1,0,0,0,0
4,5,"Warm, Sunny",1,0,0,0,0
...,...,...,...,...,...,...,...
1097,1116,NaN,0,0,0,0,0
1098,1117,NaN,0,0,0,0,0
1099,1118,NaN,0,0,0,0,0
1100,1119,NaN,0,0,0,0,0


## Transformaciones de datos para obtener dataframe final sobre el que realizar ML

### Añadimos info de tiempo a cada carrera

In [9]:
races_weather = races.merge(weather_info, how='inner', on='raceId')
races_weather

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,...,quali_date,quali_time,sprint_date,sprint_time,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,NaT,NaN,...,NaT,NaN,NaT,NaN,Sunny with temperatures reaching up to 27 °C (...,1,0,0,0,0
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,NaT,NaN,...,NaT,NaN,NaT,NaN,"Dry start, with heavy rain and thunderstorm/mo...",0,0,1,1,0
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,NaT,NaN,...,NaT,NaN,NaT,NaN,Rain,0,0,0,1,0
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,NaT,NaN,...,NaT,NaN,NaT,NaN,Sunny,1,0,0,0,0
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,NaT,NaN,...,NaT,NaN,NaT,NaN,"Warm, Sunny",1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1116,2023,19,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,...,2023-10-20,21:00:00,2023-10-21,22:00:00,NaN,0,0,0,0,0
1098,1117,2023,20,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,...,2023-10-28,21:00:00,NaT,NaN,NaN,0,0,0,0,0
1099,1118,2023,21,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,...,2023-11-03,18:00:00,2023-11-04,18:30:00,NaN,0,0,0,0,0
1100,1119,2023,22,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,...,2023-11-18,08:00:00,NaT,NaN,NaN,0,0,0,0,0


### Añadimos info de carrera a cada resultado

In [10]:
df1 = results.copy()
df1

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1.0,1,1,10.0,58,1:34:50.616,5690616.0,39.0,2.0,1:27.452,218.300,1
1,2,18,2,2,3,5,2.0,2,2,8.0,58,+5.478,5696094.0,41.0,3.0,1:27.739,217.586,1
2,3,18,3,3,7,7,3.0,3,3,6.0,58,+8.163,5698779.0,41.0,5.0,1:28.090,216.719,1
3,4,18,4,4,5,11,4.0,4,4,5.0,58,+17.181,5707797.0,58.0,7.0,1:28.603,215.464,1
4,5,18,5,1,23,3,5.0,5,5,4.0,58,+18.014,5708630.0,43.0,1.0,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25895,25901,1100,858,3,2,18,16.0,16,16,0.0,56,NaN,NaN,50.0,13.0,1:21.456,233.264,4
25896,25902,1100,825,210,20,14,17.0,17,17,0.0,52,NaN,NaN,52.0,14.0,1:21.685,232.610,3
25897,25903,1100,847,131,63,2,NaN,R,18,0.0,17,NaN,NaN,16.0,18.0,1:22.680,229.811,5
25898,25904,1100,848,3,23,8,NaN,R,19,0.0,6,NaN,NaN,6.0,19.0,1:23.349,227.966,3


In [11]:
df1 = df1.merge(races_weather, how='inner', on='raceId')
drop_list_1 = ['url',
               'fp1_date', 'fp1_time',
               'fp2_date', 'fp2_time',
               'fp3_date', 'fp3_time',
               'quali_date', 'quali_time',
               'sprint_date', 'sprint_time',
               'fastestLap', 'rank',
               'fastestLapTime', 'fastestLapSpeed',
               'time_x', 'milliseconds',
               'laps', 'statusId']
df1 = df1.drop(drop_list_1, axis=1)
df1 = df1.rename(columns={'time_y':'time'})
df1

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,circuitId,name,date,time,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1,18,1,1,22,1,1.0,1,1,10.0,...,1,Australian Grand Prix,2008-03-16,04:30:00,"Clear, dry, air temperature of 39 °C (102 °F).",0,0,0,0,0
1,2,18,2,2,3,5,2.0,2,2,8.0,...,1,Australian Grand Prix,2008-03-16,04:30:00,"Clear, dry, air temperature of 39 °C (102 °F).",0,0,0,0,0
2,3,18,3,3,7,7,3.0,3,3,6.0,...,1,Australian Grand Prix,2008-03-16,04:30:00,"Clear, dry, air temperature of 39 °C (102 °F).",0,0,0,0,0
3,4,18,4,4,5,11,4.0,4,4,5.0,...,1,Australian Grand Prix,2008-03-16,04:30:00,"Clear, dry, air temperature of 39 °C (102 °F).",0,0,0,0,0
4,5,18,5,1,23,3,5.0,5,5,4.0,...,1,Australian Grand Prix,2008-03-16,04:30:00,"Clear, dry, air temperature of 39 °C (102 °F).",0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25895,25901,1100,858,3,2,18,16.0,16,16,0.0,...,1,Australian Grand Prix,2023-04-02,05:00:00,Sunny,1,0,0,0,0
25896,25902,1100,825,210,20,14,17.0,17,17,0.0,...,1,Australian Grand Prix,2023-04-02,05:00:00,Sunny,1,0,0,0,0
25897,25903,1100,847,131,63,2,NaN,R,18,0.0,...,1,Australian Grand Prix,2023-04-02,05:00:00,Sunny,1,0,0,0,0
25898,25904,1100,848,3,23,8,NaN,R,19,0.0,...,1,Australian Grand Prix,2023-04-02,05:00:00,Sunny,1,0,0,0,0


### Añadimos datos de pilotos

In [12]:
drivers_frd = drivers.copy()

In [13]:
first_race_dates = []
results_sorted_date = df1.sort_values(by='date')
for driver in drivers_frd.driverId:
    first_race_dates.append(results_sorted_date[results_sorted_date.driverId == driver].iloc[0].date)

In [14]:
drivers_frd['firstRaceDate'] = first_race_dates
drivers_frd

,driverId,driverRef,number,code,forename,surname,dob,nationality,url,firstRaceDate
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,2007-03-18
1,2,heidfeld,<NA>,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,2000-03-12
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,2006-03-12
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,2001-03-04
4,5,kovalainen,<NA>,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen,2007-03-18
...,...,...,...,...,...,...,...,...,...,...
852,854,mick_schumacher,47,MSC,Mick,Schumacher,1999-03-22,German,http://en.wikipedia.org/wiki/Mick_Schumacher,2021-03-28
853,855,zhou,24,ZHO,Guanyu,Zhou,1999-05-30,Chinese,http://en.wikipedia.org/wiki/Guanyu_Zhou,2022-03-20
854,856,de_vries,21,DEV,Nyck,de Vries,1995-02-06,Dutch,http://en.wikipedia.org/wiki/Nyck_de_Vries,2022-09-11
855,857,piastri,81,PIA,Oscar,Piastri,2001-04-06,Australian,http://en.wikipedia.org/wiki/Oscar_Piastri,2023-03-05


In [15]:
df2 = df1.merge(drivers_frd, how='inner', on='driverId') # INNER JOIN ya que las claves coinciden
drop_list_2 = ['url',
               'number_x','number_y',
               'nationality', 'code',
               'forename', 'surname']
df2 = df2.drop(drop_list_2, axis=1)


df2['age'] = df2['date'] - df2['dob']
df2['experience'] = df2['date'] - df2['firstRaceDate']


df2

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,year,...,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driverRef,dob,firstRaceDate,age,experience
0,1,18,1,1,1,1.0,1,1,10.0,2008,...,0,0,0,0,0,hamilton,1985-01-07,2007-03-18,8469 days,364 days
1,27,19,1,1,9,5.0,5,5,4.0,2008,...,0,0,0,0,1,hamilton,1985-01-07,2007-03-18,8476 days,371 days
2,57,20,1,1,3,13.0,13,13,0.0,2008,...,0,0,1,0,0,hamilton,1985-01-07,2007-03-18,8490 days,385 days
3,69,21,1,1,5,3.0,3,3,6.0,2008,...,1,0,0,0,0,hamilton,1985-01-07,2007-03-18,8511 days,406 days
4,90,22,1,1,3,2.0,2,2,8.0,2008,...,1,0,0,0,0,hamilton,1985-01-07,2007-03-18,8525 days,420 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25895,25881,1099,858,3,20,16.0,16,16,0.0,2023,...,1,0,0,0,0,sargeant,2000-12-31,2023-03-05,8113 days,14 days
25896,25901,1100,858,3,18,16.0,16,16,0.0,2023,...,1,0,0,0,0,sargeant,2000-12-31,2023-03-05,8127 days,28 days
25897,25865,1098,857,1,18,NaN,R,20,0.0,2023,...,1,0,0,0,0,piastri,2001-04-06,2023-03-05,8003 days,0 days
25898,25880,1099,857,1,8,15.0,15,15,0.0,2023,...,1,0,0,0,0,piastri,2001-04-06,2023-03-05,8017 days,14 days


### Añadimos datos del campeonato de pilotos antes de cada carrera

In [16]:
driver_standings_mod = driver_standings.copy()
drop_list_3 = ['driverStandingsId', 'position', 'positionText', 'wins']
driver_standings_mod = driver_standings_mod.drop(drop_list_3, axis=1)
driver_standings_mod = driver_standings_mod.rename(columns={'points':'pointsAfterRace'})
driver_standings_mod

,raceId,driverId,pointsAfterRace
0,18,1,10.0
1,18,2,8.0
2,18,3,6.0
3,18,4,5.0
4,18,5,4.0
...,...,...,...
33937,1100,855,2.0
33938,1100,846,8.0
33939,1100,839,4.0
33940,1100,844,6.0


In [17]:
df3 = df2.merge(driver_standings_mod, how='left', on=['raceId', 'driverId'])
df3['driversPointsBeforeRace'] = df3['pointsAfterRace'] - df3['points']
df3

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,year,...,weather_dry,weather_wet,weather_cloudy,driverRef,dob,firstRaceDate,age,experience,pointsAfterRace,driversPointsBeforeRace
0,1,18,1,1,1,1.0,1,1,10.0,2008,...,0,0,0,hamilton,1985-01-07,2007-03-18,8469 days,364 days,10.0,0.0
1,27,19,1,1,9,5.0,5,5,4.0,2008,...,0,0,1,hamilton,1985-01-07,2007-03-18,8476 days,371 days,14.0,10.0
2,57,20,1,1,3,13.0,13,13,0.0,2008,...,1,0,0,hamilton,1985-01-07,2007-03-18,8490 days,385 days,14.0,14.0
3,69,21,1,1,5,3.0,3,3,6.0,2008,...,0,0,0,hamilton,1985-01-07,2007-03-18,8511 days,406 days,20.0,14.0
4,90,22,1,1,3,2.0,2,2,8.0,2008,...,0,0,0,hamilton,1985-01-07,2007-03-18,8525 days,420 days,28.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25895,25881,1099,858,3,20,16.0,16,16,0.0,2023,...,0,0,0,sargeant,2000-12-31,2023-03-05,8113 days,14 days,0.0,0.0
25896,25901,1100,858,3,18,16.0,16,16,0.0,2023,...,0,0,0,sargeant,2000-12-31,2023-03-05,8127 days,28 days,0.0,0.0
25897,25865,1098,857,1,18,NaN,R,20,0.0,2023,...,0,0,0,piastri,2001-04-06,2023-03-05,8003 days,0 days,0.0,0.0
25898,25880,1099,857,1,8,15.0,15,15,0.0,2023,...,0,0,0,piastri,2001-04-06,2023-03-05,8017 days,14 days,0.0,0.0


### Añadimos datos del campeonato de constructores antes de cada carrera

In [18]:
def calc_cons_points_bef_race(race, cons):
    return sum(df3.loc[(df3['raceId']==race) & (df3['constructorId']==cons)]['driversPointsBeforeRace'])

In [19]:
constructor_standings_mod = constructor_standings.copy()


constructor_standings_mod['constPointsBeforeRace'] = constructor_standings_mod.apply(
    lambda x: calc_cons_points_bef_race(x.raceId, x.constructorId), axis=1)


constructor_standings_mod = constructor_standings_mod.fillna(0)
drop_list_4 = ['constructorStandingsId', 'points', 'position', 'positionText', 'wins']
constructor_standings_mod = constructor_standings_mod.drop(drop_list_4, axis=1)
constructor_standings_mod

,raceId,constructorId,constPointsBeforeRace
0,18,1,0.0
1,18,2,0.0
2,18,3,0.0
3,18,4,0.0
4,18,5,0.0
...,...,...,...
12956,1100,214,8.0
12957,1100,3,1.0
12958,1100,213,0.0
12959,1100,210,1.0


In [20]:
df4 = df3.merge(constructor_standings_mod, how='left', on=['raceId', 'constructorId'])
df4

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,year,...,weather_wet,weather_cloudy,driverRef,dob,firstRaceDate,age,experience,pointsAfterRace,driversPointsBeforeRace,constPointsBeforeRace
0,1,18,1,1,1,1.0,1,1,10.0,2008,...,0,0,hamilton,1985-01-07,2007-03-18,8469 days,364 days,10.0,0.0,0.0
1,27,19,1,1,9,5.0,5,5,4.0,2008,...,0,1,hamilton,1985-01-07,2007-03-18,8476 days,371 days,14.0,10.0,14.0
2,57,20,1,1,3,13.0,13,13,0.0,2008,...,0,0,hamilton,1985-01-07,2007-03-18,8490 days,385 days,14.0,14.0,24.0
3,69,21,1,1,5,3.0,3,3,6.0,2008,...,0,0,hamilton,1985-01-07,2007-03-18,8511 days,406 days,20.0,14.0,28.0
4,90,22,1,1,3,2.0,2,2,8.0,2008,...,0,0,hamilton,1985-01-07,2007-03-18,8525 days,420 days,28.0,20.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25895,25881,1099,858,3,20,16.0,16,16,0.0,2023,...,0,0,sargeant,2000-12-31,2023-03-05,8113 days,14 days,0.0,0.0,1.0
25896,25901,1100,858,3,18,16.0,16,16,0.0,2023,...,0,0,sargeant,2000-12-31,2023-03-05,8127 days,28 days,0.0,0.0,1.0
25897,25865,1098,857,1,18,NaN,R,20,0.0,2023,...,0,0,piastri,2001-04-06,2023-03-05,8003 days,0 days,0.0,0.0,0.0
25898,25880,1099,857,1,8,15.0,15,15,0.0,2023,...,0,0,piastri,2001-04-06,2023-03-05,8017 days,14 days,0.0,0.0,0.0


### Eliminar columnas no necesarias y limpiar datos

In [21]:
drop_list_5 = ['resultId', 'name', 'date', 'time', 'weather', 'dob', 'firstRaceDate', 'driverRef', 'pointsAfterRace',
               'position', 'positionText', 'points', 'raceId']
df5 = df4.drop(drop_list_5, axis=1)
df5 = df5.rename(columns={'positionOrder':'position'})
df5['age'] = df5['age'].dt.days
df5['experience'] = df5['experience'].dt.days
df5['driversPointsBeforeRace'] = df5['driversPointsBeforeRace'].fillna(0)
df5['constPointsBeforeRace'] = df5['constPointsBeforeRace'].fillna(0)
df5['grid'] = df5['grid'].replace(0, 21)
df5

,driverId,constructorId,grid,position,year,round,circuitId,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,age,experience,driversPointsBeforeRace,constPointsBeforeRace
0,1,1,1,1,2008,1,1,0,0,0,0,0,8469,364,0.0,0.0
1,1,1,9,5,2008,2,2,0,0,0,0,1,8476,371,10.0,14.0
2,1,1,3,13,2008,3,3,0,0,1,0,0,8490,385,14.0,24.0
3,1,1,5,3,2008,4,4,1,0,0,0,0,8511,406,14.0,28.0
4,1,1,3,2,2008,5,5,1,0,0,0,0,8525,420,20.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25895,858,3,20,16,2023,2,77,1,0,0,0,0,8113,14,0.0,1.0
25896,858,3,18,16,2023,3,1,1,0,0,0,0,8127,28,0.0,1.0
25897,857,1,18,20,2023,1,3,1,0,0,0,0,8003,0,0.0,0.0
25898,857,1,8,15,2023,2,77,1,0,0,0,0,8017,14,0.0,0.0


In [22]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25900 entries, 0 to 25899
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   driverId                 25900 non-null  int64  
 1   constructorId            25900 non-null  int64  
 2   grid                     25900 non-null  int64  
 3   position                 25900 non-null  int64  
 4   year                     25900 non-null  int64  
 5   round                    25900 non-null  int64  
 6   circuitId                25900 non-null  int64  
 7   weather_warm             25900 non-null  int64  
 8   weather_cold             25900 non-null  int64  
 9   weather_dry              25900 non-null  int64  
 10  weather_wet              25900 non-null  int64  
 11  weather_cloudy           25900 non-null  int64  
 12  age                      25900 non-null  int64  
 13  experience               25900 non-null  int64  
 14  driversPointsBeforeRac

### One-hot encoding

In [23]:
# One-hot driverId 
driver_onehot = pd.get_dummies(df5['driverId'], prefix='driverId')

# One-hot constructorId 
constructor_onehot = pd.get_dummies(df5['constructorId'], prefix='constructorId')

# One-hot circuitId 
circuit_onehot = pd.get_dummies(df5['circuitId'], prefix='circuitId')

In [24]:
df6 = pd.DataFrame()
df6 = df5[['driverId', 'constructorId', 'circuitId']]
df6 = df6.join(driver_onehot)
df6 = df6.join(constructor_onehot)
df6 = df6.join(circuit_onehot)
df6 = df6.drop(['driverId', 'constructorId', 'circuitId'], axis=1)
df6 = df6.join(df5[['grid', 'position', 'year', 'round', 'weather_warm', 'weather_cold', 'weather_dry', 'weather_wet', 
                    'weather_cloudy', 'age', 'experience', 'driversPointsBeforeRace', 'constPointsBeforeRace']])
df6

,driverId_1,driverId_2,driverId_3,driverId_4,driverId_5,driverId_6,driverId_7,driverId_8,driverId_9,driverId_10,...,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,age,experience,driversPointsBeforeRace,constPointsBeforeRace
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,8469,364,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,1,8476,371,10.0,14.0
2,1,0,0,0,0,0,0,0,0,0,...,3,0,0,1,0,0,8490,385,14.0,24.0
3,1,0,0,0,0,0,0,0,0,0,...,4,1,0,0,0,0,8511,406,14.0,28.0
4,1,0,0,0,0,0,0,0,0,0,...,5,1,0,0,0,0,8525,420,20.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25895,0,0,0,0,0,0,0,0,0,0,...,2,1,0,0,0,0,8113,14,0.0,1.0
25896,0,0,0,0,0,0,0,0,0,0,...,3,1,0,0,0,0,8127,28,0.0,1.0
25897,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,8003,0,0.0,0.0
25898,0,0,0,0,0,0,0,0,0,0,...,2,1,0,0,0,0,8017,14,0.0,0.0


# Guardar en CSV para pasar a ML

In [25]:
df5.to_csv('data_ready/LEARNING_DF_NORMAL.csv', index=False)
df6.to_csv('data_ready/LEARNING_DF_ONEHOT.csv', index=False)

In [3]:
constructors

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
...,...,...,...,...,...
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport
207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team
209,213,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri
